<a href="https://colab.research.google.com/github/arielwendichansky/DI_Bootcamp/blob/master/Week7/Day2/Exercise_XP/Exercise_XP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install selenium

#🌟 Exercise 3 : Scrape Dynamic Content From Rotten Tomatoes
Task:
Use Selenium to navigate to the Rotten Tomatoes Certified Fresh Movies page.
Extract the HTML content after it’s fully loaded.
Use BeautifulSoup to parse and extract the movie titles, scores, and release dates.
Instructions
Set up Selenium WebDriver and navigate to the Rotten Tomatoes page.
Extract the HTML content using driver.page_source.
Parse the HTML with BeautifulSoup.
Find and extract the desired movie information.
Print the extracted data

In [107]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pprint  # To tidy up
import pandas as pd
from datetime import datetime
import time
from collections import Counter

In [ ]:
# Configure the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to avoid UI pop-up
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# Initialize the WebDriver
driver = webdriver.Chrome(options=options)
try:
    # Target URL
    url = "https://www.rottentomatoes.com/browse/movies_at_home/critics:certified_fresh"
    # Navigate to the page
    driver.get(url)

    # Get page source and prepare it for parsing
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    # Locate the container holding all the flex containers
    grid_container = soup.find('div', class_='discovery-tiles')
    # Find all flex-container elements within the grid container
    flex_containers = grid_container.find_all('div', class_='flex-container')
      # Initialize an empty list to store movie data dictionaries
    movie_data = []

    # Loop through each flex-container to extract the desired span elements
    for flex in flex_containers:
        name = flex.find('span', class_='p--small').text.strip()
        year = flex.find('span', class_='smaller').text.strip()[10:]
        score = flex.find('score-pairs-deprecated')['audiencescore']

        # Store the movie data in a dictionary
        data = {'Name': name, 'Year': year, 'Score': score}
        movie_data.append(data)

    # Create a DataFrame from the movie data list
    df = pd.DataFrame(movie_data)


    print(df)

finally:
    # Close the driver
    driver.quit()

                      Name          Year Score
0      Hundreds of Beavers  Apr 15, 2024    94
1     The Zone of Interest  Feb 20, 2024    78
2                 One Life   Apr 9, 2024    95
3              Poor Things  Feb 27, 2024    79
4        Anatomy of a Fall  Dec 22, 2023    90
5     You'll Never Find Me  Mar 22, 2024    72
6           Coup de Chance  Apr 12, 2024    79
7            The Gentlemen  Mar 24, 2020    84
8      All of Us Strangers  Feb 22, 2024    91
9                    Wonka  Jan 30, 2024    91
10          Dream Scenario  Dec 22, 2023    68
11                    Dune  Oct 22, 2021    90
12             Oppenheimer  Nov 21, 2023    91
13       Godzilla vs. Kong  Mar 31, 2021    91
14              Talk to Me  Sep 12, 2023    82
15            Megan Leavey  Jul 10, 2017    82
16                   Ennio   Apr 9, 2024   100
17                   Split   Apr 5, 2017    79
18            Perfect Days   Mar 5, 2024    89
19             The Fallout  Jan 27, 2022    84
20         Or

#🌟 Exercise 4 : Scrape And Categorize News Articles From A JavaScript-Enabled News Site

Visit a section of a news website (e.g., the Technology section of BBC News).

Scrape news article titles and their publication dates.

Categorize articles based on their publication month.




In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Configure the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to avoid UI pop-up
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

 # Target URL
url = "https://www.jpost.com/"  # Replace with your URL
# Navigate to the page
driver.get(url)

# Find the element by link text
element = driver.find_element(By.LINK_TEXT, 'Kabbalah')

# Click on the element
element.click()
try:
    wait = WebDriverWait(driver, 10)

    # Get page source and prepare it for parsing
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the desired elements within the page
    name_list = soup.find_all('h3', class_="itc-info-title")
    names = [name.text.strip() for name in name_list]

    date_list = soup.find_all('span', class_="category-list-item-date")
    dates = [date.text.strip() for date in date_list]

    print(names)
    print(dates)
finally:
    # Close the driver
    driver.quit()


['Shabbat - Rehearsal of The Return', 'The First Creation Of Adam: Can we make G-d work?', 'LIVE: Dr. Eduard Shyfrin discusses his upcoming book “The Relativity of Death”', 'First creation of Adam: Does G-d use Ockham’s Razor?', 'The first creation of Adam  - Where, When, Why?', 'The Red Button of Adam - Part 2', 'The ‘Red Button’ of Adam - Part 1', 'Eduard Shyfrin speaks at Jewish studies conference in Jerusalem', 'Part 6 - Relativity of Death: The origin of information', 'Part 8 - Relativity of Death: Life and death inside a Black Hole – opinion']
['31/05/2023', '04/04/2023', '15/03/2023', '02/02/2023', '12/12/2022', '02/10/2022', '30/08/2022', '09/08/2022', '26/06/2022', '26/06/2022']


In [40]:
date_format = "%d/%m/%Y"

articles_by_month = {
    'January': [],
    'February': [],
    'March': [],
    'April': [],
    'May': [],
    'June': [],
    'July': [],
    'August': [],
    'September': [],
    'October': [],
    'November': [],
    'December': []
}

for date, name in zip(dates, names):
    # Convert the date string to a datetime object
    date_object = datetime.strptime(date, date_format)

    month_name = date_object.strftime('%B')

    articles_by_month[month_name].append(name)


for month, articles in articles_by_month.items():
    print(month)
    for article in articles:
        print("- " + article)
    print()

January

February
- First creation of Adam: Does G-d use Ockham’s Razor?

March
- LIVE: Dr. Eduard Shyfrin discusses his upcoming book “The Relativity of Death”

April
- The First Creation Of Adam: Can we make G-d work?

May
- Shabbat - Rehearsal of The Return

June
- Part 6 - Relativity of Death: The origin of information
- Part 8 - Relativity of Death: Life and death inside a Black Hole – opinion

July

August
- The ‘Red Button’ of Adam - Part 1
- Eduard Shyfrin speaks at Jewish studies conference in Jerusalem

September

October
- The Red Button of Adam - Part 2

November

December
- The first creation of Adam  - Where, When, Why?



# 🌟 Exercise 5 : Scrape And Analyze Weather Data From A JavaScript-Enabled Weather Website

Visit a specific city’s weather forecast page on a weather website.

Scrape weather forecast data including temperature, condition, and humidity.

Analyze the data to find the average temperature and most common weather condition.

In [98]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# Configure the Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run headless to avoid UI pop-up
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

 # Target URL
url = "https://weather.com/weather/today/l/5654c9edb068447e18017135241def26b619dac3428aa24e38e23a11d051991a"
# Navigate to the page
driver.get(url)

# Wait for the search bar element to be present
search_bar = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, '10 Day')))
element = driver.find_element(By.LINK_TEXT, '10 Day')

element.click()


try:
    wait = WebDriverWait(driver, 10)

    # Get page source and prepare it for parsing
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the desired elements within the page
    temperature_week = soup.find_all('span',class_="DetailsSummary--highTempValue--3PjlX")
    temperatures = [t.text.strip() for t in temperature_week]

    condition_week = soup.find_all('span', class_="DetailsSummary--extendedData--307Ax")
    condition = [c.text.strip() for c in condition_week]

    humidity_week = soup.find_all('span', {'data-testid': 'PercentageValue'})
    humidity = [h.text.strip() for h in humidity_week][0:15]

    print(temperatures)
    print(condition)
    print(humidity)
finally:
    # Close the driver
    driver.quit()

['--', '77°', '74°', '75°', '75°', '77°', '86°', '93°', '86°', '81°', '79°', '79°', '79°', '79°', '80°']
['Mostly Clear', 'Sunny', 'Partly Cloudy', 'Mostly Sunny', 'Mostly Sunny', 'Sunny', 'Mostly Sunny', 'Partly Cloudy', 'Mostly Cloudy', 'Partly Cloudy', 'Mostly Sunny', 'Mostly Sunny', 'Sunny', 'Partly Cloudy', 'Partly Cloudy']
['0%', '38%', '4%', '4%', '63%', '9%', '82%', '16%', '16%', '67%', '8%', '79%', '6%', '6%', '63%']


In [ ]:
int_list = []
for item in temperatures:

    cleaned_item = ''.join(filter(str.isdigit, item))
    if cleaned_item:
        int_list.append(int(cleaned_item))
    else:
        int_list.append(None)
print(int_list)

In [ ]:
int_list_humidity = []
for item in humidity:

    cleaned_item = ''.join(filter(str.isdigit, item))
    if cleaned_item:
        int_list_humidity.append(int(cleaned_item))
    else:
        int_list_humidity.append(None)
print(int_list_humidity)

In [112]:
data = {'Temperature':int_list,
        'Condition': condition,
        'Humidity': int_list_humidity
}

df = pd.DataFrame(data)
df

weather_counts = Counter(condition)
most_common_weather = weather_counts.most_common(1)


# Print the result
print('The average temperature in Tel Aviv for the following 10 days is ', df['Temperature'].mean(), ' F')
print('The the most common weather condition for in Tel Aviv the following 10 days is ',most_common_weather[0][0])


The average temperature in Tel Aviv for the following 10 days is  80.0  F
The the most common weather condition for in Tel Aviv the following 10 days is  Partly Cloudy
